In [ ]:
import cvxpy as cp
import numpy as np

# Define dimension
n = 2  # Qubit space (2x2 matrices)
num_M = 4  # Number of measurement operators

# Define SDP variables (each M_k is a 2x2 positive semidefinite matrix)
M = [cp.Variable((n, n), hermitian=True) for _ in range(num_M)]
t = cp.Variable((num_M, 2))  # Auxiliary variables for absolute value constraints

# Define basis states
ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])
ket_plus = (ket_0 + ket_1) / np.sqrt(2)
ket_minus = (ket_0 - ket_1) / np.sqrt(2)

# Compute expectation values
states = [[ket_0, ket_1], [ket_plus, ket_minus]]
expectations = [[cp.real(cp.quad_form(states[u][b], M[k])) for u in range(2)] for k in range(num_M) for b in range(2)]

# Constraints
constraints = [sum(M) == np.eye(n)]  # POVM condition
for k in range(num_M):
    constraints.append(M[k] >> 0)  # Positive semidefinite condition
    for u in range(2):
        diff = expectations[k][u] - expectations[k][1 - u]
        constraints.append(diff <= t[k, u])
        constraints.append(-diff <= t[k, u])

# Objective function
objective = cp.Minimize(0.5 * cp.sum(t))

# Solve SDP
problem = cp.Problem(objective, constraints)
problem.solve()

# Output results
print("Optimal value:", problem.value)
for k in range(num_M):
    print(f"M_{k+1}:\n", M[k].value)
